### Reference
https://github.com/GzyAftermath/DATM/tree/main

# helper function and import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1513/Project_B_Supp/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ECE1513/Project_B_Supp


In [ ]:
!pip install kornia

In [ ]:
!pip install wandb

In [ ]:
import os
import sys

import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils
from tqdm import tqdm
import wandb
import copy
import random
import kornia as K



random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [ ]:
import time
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from scipy.ndimage.interpolation import rotate as scipyrotate
from networks import MLP, ConvNet, LeNet, AlexNet, VGG11BN, VGG11, ResNet18, ResNet18BN_AP, ResNet18_AP, ResNet18BN


from torch.utils.data import Dataset, DataLoader
from PIL import Image

def get_dataset(dataset, data_path, batch_size=1, subset="imagenette", args=None, baseline=False):

    class_map = None
    loader_train_dict = None
    class_map_inv = None

    if dataset == 'CIFAR10':
        channel = 3
        im_size = (32, 32)
        num_classes = 10
        mean = [0.4914, 0.4822, 0.4465]
        std = [0.2023, 0.1994, 0.2010]
        if baseline:
            transform =transforms.Compose(
            [
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ])
        elif args.zca:
            transform = transforms.Compose([transforms.ToTensor()])
        else:
            transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
        dst_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transform) # no augmentation
        dst_test = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)
        class_names = dst_train.classes
        class_map = {x:x for x in range(num_classes)}

    elif dataset == 'MNIST':
        channel = 1
        im_size = (28, 28)
        num_classes = 10
        mean = [0.1307]
        std = [0.3081]
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
        dst_train = datasets.MNIST(data_path, train=True, download=True, transform=transform) # no augmentation
        dst_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)
        class_names = [str(c) for c in range(num_classes)]
        class_map = {x:x for x in range(num_classes)}

    elif dataset == 'MHIST':
        im_size = (32, 32)
        num_classes=2
        channel=3
        mean = [0,0,0]
        std = [0,0,0]
        #train_dir = './mhist_dataset/train'
        train_dir = './mhist_dataset/augmentation'
        test_dir = './mhist_dataset/test'

        mhist_data = datasets.ImageFolder(root=train_dir, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]))
        #mean, std = compute_mean_std(mhist_data)

        transform_mhist = transforms.Compose([
            transforms.Resize(im_size),
            transforms.ToTensor(),
            #transforms.Normalize(mean=mean, std=std)
        ])
        dst_train = datasets.ImageFolder(root=train_dir, transform=transform_mhist)
        dst_test = datasets.ImageFolder(root=test_dir, transform=transform_mhist)
        class_names = [str(c) for c in range(num_classes)]
        class_map = {x:x for x in range(num_classes)}

    else:
        exit('unknown dataset: %s'%dataset)



    if args.zca:
        images = []
        labels = []
        print("Train ZCA")
        for i in tqdm(range(len(dst_train))):
            im, lab = dst_train[i]
            images.append(im)
            labels.append(lab)
        images = torch.stack(images, dim=0).to(args.device)
        labels = torch.tensor(labels, dtype=torch.long, device="cpu")
        zca = K.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
        zca.fit(images)
        zca_images = zca(images).to("cpu")
        dst_train = TensorDataset(zca_images, labels)

        images = []
        labels = []
        print("Test ZCA")
        for i in tqdm(range(len(dst_test))):
            im, lab = dst_test[i]
            images.append(im)
            labels.append(lab)
        images = torch.stack(images, dim=0).to(args.device)
        labels = torch.tensor(labels, dtype=torch.long, device="cpu")

        zca_images = zca(images).to("cpu")
        dst_test = TensorDataset(zca_images, labels)

        print(type(zca))


        args.zca_trans = zca


    testloader = torch.utils.data.DataLoader(dst_test, batch_size=256, shuffle=False, num_workers=2)


    return channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader, loader_train_dict, class_map, class_map_inv



<ipython-input-29-635b7ad7382a>:4: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import rotate as scipyrotate


In [ ]:
def get_network(model, channel, num_classes, im_size=(32, 32), dist=True):
    torch.random.manual_seed(int(time.time() * 1000) % 100000)
    net_width, net_depth, net_act, net_norm, net_pooling = get_default_convnet_setting()

    if model == 'MLP':
        net = MLP(channel=channel, num_classes=num_classes)
    elif model == 'MLP_Tiny':
        net = MLP(channel=channel, num_classes=num_classes,res=64)
    elif model == 'ConvNet':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'LeNet':
        net = LeNet(channel=channel, num_classes=num_classes)
    elif model == 'LeNet_Tiny':
        net = LeNet(channel=channel, num_classes=num_classes, res=64)
    elif model == 'AlexNet':
        net = AlexNet(channel=channel, num_classes=num_classes)
    elif model == 'AlexNet_Tiny':
        net = AlexNet(channel=channel, num_classes=num_classes, res=64)
    elif model == 'VGG11':
        net = VGG11( channel=channel, num_classes=num_classes)
    elif model == 'VGG11_Tiny':
        net = VGG11_Tiny( channel=channel, num_classes=num_classes)
    elif model == 'VGG11BN':
        net = VGG11BN(channel=channel, num_classes=num_classes)
    elif model == 'ResNet18':
        net = ResNet18(channel=channel, num_classes=num_classes)
    elif model == 'ResNet18BN_AP':
        net = ResNet18BN_AP(channel=channel, num_classes=num_classes)
    elif model == 'ResNet18_AP':
        net = ResNet18_AP(channel=channel, num_classes=num_classes)
    elif model == 'ResNet18BN':
        net = ResNet18BN(channel=channel, num_classes=num_classes)
    elif model == 'ResNet18_Tiny':
        net = ResNet18_Tiny(channel=channel, num_classes=num_classes)
    elif model == 'ResNet18BN_Tiny':
        net = ResNet18BN_Tiny(channel=channel, num_classes=num_classes)
    elif model == 'ConvNetD1':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=1, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD2':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=2, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD3':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=3, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD4':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=4, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD4BN':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=4, net_act=net_act, net_norm='batchnorm', net_pooling=net_pooling, im_size=im_size)

    elif model == 'ConvNetD5':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=5, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD6':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=6, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD7':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=7, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)
    elif model == 'ConvNetD8':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=8, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling, im_size=im_size)


    elif model == 'ConvNetW32':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=32, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetW64':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=64, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetW128':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=128, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetW256':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=256, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetW512':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=512, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetW1024':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=1024, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling=net_pooling)

    elif model == "ConvNetKIP":
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=1024, net_depth=net_depth, net_act=net_act,
                      net_norm="none", net_pooling=net_pooling)

    elif model == 'ConvNetAS':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act='sigmoid', net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetAR':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act='relu', net_norm=net_norm, net_pooling=net_pooling)
    elif model == 'ConvNetAL':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act='leakyrelu', net_norm=net_norm, net_pooling=net_pooling)

    elif model == 'ConvNetNN':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm='none', net_pooling=net_pooling)
    elif model == 'ConvNetBN':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm='batchnorm', net_pooling=net_pooling)
    elif model == 'ConvNetLN':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm='layernorm', net_pooling=net_pooling)
    elif model == 'ConvNetIN':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm='instancenorm', net_pooling=net_pooling)
    elif model == 'ConvNetGN':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm='groupnorm', net_pooling=net_pooling)

    elif model == 'ConvNetNP':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling='none')
    elif model == 'ConvNetMP':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling='maxpooling')
    elif model == 'ConvNetAP':
        net = ConvNet(channel=channel, num_classes=num_classes, net_width=net_width, net_depth=net_depth, net_act=net_act, net_norm=net_norm, net_pooling='avgpooling')


    else:
        net = None
        exit('DC error: unknown model')

    if dist:
        gpu_num = torch.cuda.device_count()
        if gpu_num>0:
            device = 'cuda'
            if gpu_num>1:
                net = nn.DataParallel(net)
        else:
            device = 'cpu'
        net = net.to(device)

    return net

In [ ]:
def get_eval_pool(eval_mode, model, model_eval):
    if eval_mode == 'M': # multiple architectures
        # model_eval_pool = ['MLP', 'ConvNet', 'AlexNet', 'VGG11', 'ResNet18', 'LeNet']
        model_eval_pool = ['ConvNet', 'AlexNet', 'VGG11', 'ResNet18']
        # model_eval_pool = ['MLP', 'ConvNet', 'AlexNet', 'VGG11', 'ResNet18']
    elif eval_mode == 'W': # ablation study on network width
        model_eval_pool = ['ConvNetW32', 'ConvNetW64', 'ConvNetW128', 'ConvNetW256']
    elif eval_mode == 'D': # ablation study on network depth
        model_eval_pool = ['ConvNetD1', 'ConvNetD2', 'ConvNetD3', 'ConvNetD4']
    elif eval_mode == 'A': # ablation study on network activation function
        model_eval_pool = ['ConvNetAS', 'ConvNetAR', 'ConvNetAL']
    elif eval_mode == 'P': # ablation study on network pooling layer
        model_eval_pool = ['ConvNetNP', 'ConvNetMP', 'ConvNetAP']
    elif eval_mode == 'N': # ablation study on network normalization layer
        model_eval_pool = ['ConvNetNN', 'ConvNetBN', 'ConvNetLN', 'ConvNetIN', 'ConvNetGN']
    elif eval_mode == 'S': # itself
        model_eval_pool = [model[:model.index('BN')]] if 'BN' in model else [model]
    elif eval_mode == 'C':
        model_eval_pool = [model, 'ConvNet']
    elif eval_mode == 'BN':
        model_eval_pool = ['ConvNet','ConvNetBN','ResNet18','ResNet18BN','AlexNet', 'VGG11', 'ResNet18_AP']
    else:
        model_eval_pool = [model_eval]
    return model_eval_pool

In [ ]:
def epoch(mode, dataloader, net, optimizer, criterion, args, aug, texture=False, If_Float = False):
    loss_avg, acc_avg, num_exp = 0, 0, 0
    if args.parall_eva==False:
        device = torch.device("cuda:0")
    else:
        device = args.device


    if mode == 'train':
        net.train()
    else:
        net.eval()
    net = net.to(device)
    for i_batch, datum in enumerate(dataloader):
        img = datum[0].float().to(device)
        if If_Float:
            lab = datum[1].float().to(device)
        else:
            lab = datum[1].long().to(device)
        if aug:
            if args.dsa:
                img = DiffAugment(img, args.dsa_strategy, param=args.dsa_param)
            else:
                img = augment(img, args.dc_aug_param, device=device)
        img = img.to(device)
        if args.dataset == "ImageNet" and mode != "train":
            lab = torch.tensor([class_map[x.item()] for x in lab]).to(device)

        n_b = lab.shape[0]

        output = net(img)
        #print(lab)
        loss = criterion(output, lab)


        if If_Float:
            acc = 1.
        else:
            acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))

        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b

        if mode == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [ ]:
def evaluate_synset(it_eval, net, images_train, labels_train, testloader, args, return_loss=False, texture=False, train_criterion=None, Preciser_Scheduler=False, type=1):
    if args.parall_eva==False:
        device = torch.device("cuda:0")
    else:
        device = args.device
    net = net.to(device)
    images_train.to(device)
    labels_train.to(device)
    lr = float(args.lr_net)
    Epoch = int(args.epoch_eval_train)

    # if Preciser_Scheduler:
    #     LR_begin=0.0000000001
    #     LR_End = float(args.lr_net)
    #     if type==0:
    #         t=0
    #     else:
    #         t=500
    #     T=Epoch
    #     lambda1 = lambda epoch: ((LR_End-LR_begin)*epoch / t) if epoch < t else  LR_End * (1+math.cos(math.pi*(epoch - t)/(T-t)))/2.
    #     optimizer = torch.optim.Adam(net.parameters(), lr=LR_End, weight_decay=0.0005)
    #     scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
    #else:
    lr_schedule = [Epoch//2+1]
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)

    '''test'''
    test_criterion = nn.CrossEntropyLoss().to(device)
    If_Float = True
    if train_criterion == None:
        train_criterion = nn.CrossEntropyLoss().to(device)
        If_Float = False

    dst_train = TensorDataset(images_train, labels_train)
    trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)

    start = time.time()
    acc_train_list = []
    loss_train_list = []

    for ep in tqdm(range(Epoch+1)):
        loss_train, acc_train = epoch('train', trainloader, net, optimizer, train_criterion, args, aug=True, texture=texture, If_Float=If_Float)
        acc_train_list.append(acc_train)
        loss_train_list.append(loss_train)
        if ep == Epoch:
            with torch.no_grad():
                loss_test, acc_test = epoch('test', testloader, net, optimizer, test_criterion, args, aug=False, If_Float = False)
        #if Preciser_Scheduler:
        #    scheduler.step()
        #else:
        if ep in lr_schedule:
            lr *= 0.1
            optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)


    time_train = time.time() - start

    print('%s Evaluate_%02d: epoch = %04d train time = %d s train loss = %.6f train acc = %.4f, test acc = %.4f' % (get_time(), it_eval, Epoch, int(time_train), loss_train, acc_train, acc_test))

    if return_loss:
        return net, acc_train_list, acc_test, loss_train_list, loss_test
    else:
        return net, acc_train_list, acc_test

In [ ]:
class TensorDataset(Dataset):
    def __init__(self, images, labels): # images: n x c x h x w tensor
        self.images = images.detach().float()
        self.labels = labels.detach()

    def __getitem__(self, index):
        return self.images[index], self.labels[index]

    def __len__(self):
        return self.images.shape[0]

In [ ]:
def get_default_convnet_setting():
    net_width, net_depth, net_act, net_norm, net_pooling = 128, 3, 'relu', 'instancenorm', 'avgpooling'
    return net_width, net_depth, net_act, net_norm, net_pooling


In [ ]:
def get_time():
    return str(time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()))

In [ ]:
def rand_scale(x, param):
    # x>1, max scale
    # sx, sy: (0, +oo), 1: orignial size, 0.5: enlarge 2 times
    ratio = param.ratio_scale
    set_seed_DiffAug(param)
    sx = torch.rand(x.shape[0]) * (ratio - 1.0/ratio) + 1.0/ratio
    set_seed_DiffAug(param)
    sy = torch.rand(x.shape[0]) * (ratio - 1.0/ratio) + 1.0/ratio
    theta = [[[sx[i], 0,  0],
            [0,  sy[i], 0],] for i in range(x.shape[0])]
    theta = torch.tensor(theta, dtype=torch.float)
    if param.batchmode: # batch-wise:
        theta[:] = theta[0]
    grid = F.affine_grid(theta, x.shape, align_corners=True).to(x.device)
    x = F.grid_sample(x, grid, align_corners=True)
    return x


def rand_rotate(x, param): # [-180, 180], 90: anticlockwise 90 degree
    ratio = param.ratio_rotate
    set_seed_DiffAug(param)
    theta = (torch.rand(x.shape[0]) - 0.5) * 2 * ratio / 180 * float(np.pi)
    theta = [[[torch.cos(theta[i]), torch.sin(-theta[i]), 0],
        [torch.sin(theta[i]), torch.cos(theta[i]),  0],]  for i in range(x.shape[0])]
    theta = torch.tensor(theta, dtype=torch.float)
    if param.batchmode: # batch-wise:
        theta[:] = theta[0]
    grid = F.affine_grid(theta, x.shape, align_corners=True).to(x.device)
    x = F.grid_sample(x, grid, align_corners=True)
    return x


def rand_flip(x, param):
    prob = param.prob_flip
    set_seed_DiffAug(param)
    randf = torch.rand(x.size(0), 1, 1, 1, device=x.device)
    if param.batchmode: # batch-wise:
        randf[:] = randf[0]
    return torch.where(randf < prob, x.flip(3), x)


def rand_brightness(x, param):
    ratio = param.brightness
    set_seed_DiffAug(param)
    randb = torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device)
    if param.batchmode:  # batch-wise:
        randb[:] = randb[0]
    x = x + (randb - 0.5)*ratio
    return x


def rand_saturation(x, param):
    ratio = param.saturation
    x_mean = x.mean(dim=1, keepdim=True)
    set_seed_DiffAug(param)
    rands = torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device)
    if param.batchmode:  # batch-wise:
        rands[:] = rands[0]
    x = (x - x_mean) * (rands * ratio) + x_mean
    return x


def rand_contrast(x, param):
    ratio = param.contrast
    x_mean = x.mean(dim=[1, 2, 3], keepdim=True)
    set_seed_DiffAug(param)
    randc = torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device)
    if param.batchmode:  # batch-wise:
        randc[:] = randc[0]
    x = (x - x_mean) * (randc + ratio) + x_mean
    return x


def rand_crop(x, param):
    # The image is padded on its surrounding and then cropped.
    ratio = param.ratio_crop_pad
    shift_x, shift_y = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    set_seed_DiffAug(param)
    translation_x = torch.randint(-shift_x, shift_x + 1, size=[x.size(0), 1, 1], device=x.device)
    set_seed_DiffAug(param)
    translation_y = torch.randint(-shift_y, shift_y + 1, size=[x.size(0), 1, 1], device=x.device)
    if param.batchmode:  # batch-wise:
        translation_x[:] = translation_x[0]
        translation_y[:] = translation_y[0]
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(x.size(2), dtype=torch.long, device=x.device),
        torch.arange(x.size(3), dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + translation_x + 1, 0, x.size(2) + 1)
    grid_y = torch.clamp(grid_y + translation_y + 1, 0, x.size(3) + 1)
    x_pad = F.pad(x, [1, 1, 1, 1, 0, 0, 0, 0])
    x = x_pad.permute(0, 2, 3, 1).contiguous()[grid_batch, grid_x, grid_y].permute(0, 3, 1, 2)
    return x


def rand_cutout(x, param):
    ratio = param.ratio_cutout
    cutout_size = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    set_seed_DiffAug(param)
    offset_x = torch.randint(0, x.size(2) + (1 - cutout_size[0] % 2), size=[x.size(0), 1, 1], device=x.device)
    set_seed_DiffAug(param)
    offset_y = torch.randint(0, x.size(3) + (1 - cutout_size[1] % 2), size=[x.size(0), 1, 1], device=x.device)
    if param.batchmode:  # batch-wise:
        offset_x[:] = offset_x[0]
        offset_y[:] = offset_y[0]
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(cutout_size[0], dtype=torch.long, device=x.device),
        torch.arange(cutout_size[1], dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + offset_x - cutout_size[0] // 2, min=0, max=x.size(2) - 1)
    grid_y = torch.clamp(grid_y + offset_y - cutout_size[1] // 2, min=0, max=x.size(3) - 1)
    mask = torch.ones(x.size(0), x.size(2), x.size(3), dtype=x.dtype, device=x.device)
    mask[grid_batch, grid_x, grid_y] = 0
    x = x * mask.unsqueeze(1)
    return x


In [ ]:
AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'crop': [rand_crop],
    'cutout': [rand_cutout],
    'flip': [rand_flip],
    'scale': [rand_scale],
    'rotate': [rand_rotate],
}

In [ ]:
def set_seed_DiffAug(param):
    if param.latestseed == -1:
        return
    else:
        torch.random.manual_seed(param.latestseed)
        param.latestseed += 1

In [ ]:
def DiffAugment(x, strategy='', seed = -1, param = None):
    if seed == -1:
        param.batchmode = False
    else:
        param.batchmode = True

    param.latestseed = seed

    if strategy == 'None' or strategy == 'none':
        return x

    if strategy:
        if param.aug_mode == 'M': # original
            for p in strategy.split('_'):
                for f in AUGMENT_FNS[p]:
                    x = f(x, param)
        elif param.aug_mode == 'S':
            pbties = strategy.split('_')
            set_seed_DiffAug(param)
            p = pbties[torch.randint(0, len(pbties), size=(1,)).item()]
            for f in AUGMENT_FNS[p]:
                x = f(x, param)
        else:
            exit('Error ZH: unknown augmentation mode.')
        x = x.contiguous()
    return x

In [ ]:
class ParamDiffAug():
    def __init__(self):
        self.aug_mode = 'S' #'multiple or single'
        self.prob_flip = 0.5
        self.ratio_scale = 1.2
        self.ratio_rotate = 15.0
        self.ratio_crop_pad = 0.125
        self.ratio_cutout = 0.5 # the size would be 0.5x0.5
        self.ratio_noise = 0.05
        self.brightness = 1.0
        self.saturation = 2.0
        self.contrast = 0.5

In [ ]:
from collections import namedtuple
from contextlib import contextmanager
import types

class ReparamModule(nn.Module):
    def _get_module_from_name(self, mn):
        if mn == '':
            return self
        m = self
        for p in mn.split('.'):
            m = getattr(m, p)
        return m

    def __init__(self, module):
        super(ReparamModule, self).__init__()
        self.module = module

        param_infos = []  # (module name/path, param name)
        shared_param_memo = {}
        shared_param_infos = []  # (module name/path, param name, src module name/path, src param_name)
        params = []
        param_numels = []
        param_shapes = []
        for mn, m in self.named_modules():
            for n, p in m.named_parameters(recurse=False):
                if p is not None:
                    if p in shared_param_memo:
                        shared_mn, shared_n = shared_param_memo[p]
                        shared_param_infos.append((mn, n, shared_mn, shared_n))
                    else:
                        shared_param_memo[p] = (mn, n)
                        param_infos.append((mn, n))
                        params.append(p.detach())
                        param_numels.append(p.numel())
                        param_shapes.append(p.size())

        assert len(set(p.dtype for p in params)) <= 1, \
            "expects all parameters in module to have same dtype"

        # store the info for unflatten
        self._param_infos = tuple(param_infos)
        self._shared_param_infos = tuple(shared_param_infos)
        self._param_numels = tuple(param_numels)
        self._param_shapes = tuple(param_shapes)

        # flatten
        flat_param = nn.Parameter(torch.cat([p.reshape(-1) for p in params], 0))
        self.register_parameter('flat_param', flat_param)
        self.param_numel = flat_param.numel()
        del params
        del shared_param_memo

        # deregister the names as parameters
        for mn, n in self._param_infos:
            delattr(self._get_module_from_name(mn), n)
        for mn, n, _, _ in self._shared_param_infos:
            delattr(self._get_module_from_name(mn), n)

        # register the views as plain attributes
        self._unflatten_param(self.flat_param)

        # now buffers
        # they are not reparametrized. just store info as (module, name, buffer)
        buffer_infos = []
        for mn, m in self.named_modules():
            for n, b in m.named_buffers(recurse=False):
                if b is not None:
                    buffer_infos.append((mn, n, b))

        self._buffer_infos = tuple(buffer_infos)
        self._traced_self = None

    def trace(self, example_input, **trace_kwargs):
        assert self._traced_self is None, 'This ReparamModule is already traced'

        if isinstance(example_input, torch.Tensor):
            example_input = (example_input,)
        example_input = tuple(example_input)
        example_param = (self.flat_param.detach().clone(),)
        example_buffers = (tuple(b.detach().clone() for _, _, b in self._buffer_infos),)

        self._traced_self = torch.jit.trace_module(
            self,
            inputs=dict(
                _forward_with_param=example_param + example_input,
                _forward_with_param_and_buffers=example_param + example_buffers + example_input,
            ),
            **trace_kwargs,
        )

        # replace forwards with traced versions
        self._forward_with_param = self._traced_self._forward_with_param
        self._forward_with_param_and_buffers = self._traced_self._forward_with_param_and_buffers
        return self

    def clear_views(self):
        for mn, n in self._param_infos:
            setattr(self._get_module_from_name(mn), n, None)  # This will set as plain attr

    def _apply(self, *args, **kwargs):
        if self._traced_self is not None:
            self._traced_self._apply(*args, **kwargs)
            return self
        return super(ReparamModule, self)._apply(*args, **kwargs)

    def _unflatten_param(self, flat_param):
        ps = (t.view(s) for (t, s) in zip(flat_param.split(self._param_numels), self._param_shapes))
        for (mn, n), p in zip(self._param_infos, ps):
            setattr(self._get_module_from_name(mn), n, p)  # This will set as plain attr
        for (mn, n, shared_mn, shared_n) in self._shared_param_infos:
            setattr(self._get_module_from_name(mn), n, getattr(self._get_module_from_name(shared_mn), shared_n))

    @contextmanager
    def unflattened_param(self, flat_param):
        saved_views = [getattr(self._get_module_from_name(mn), n) for mn, n in self._param_infos]
        self._unflatten_param(flat_param)
        yield
        # Why not just `self._unflatten_param(self.flat_param)`?
        # 1. because of https://github.com/pytorch/pytorch/issues/17583
        # 2. slightly faster since it does not require reconstruct the split+view
        #    graph
        for (mn, n), p in zip(self._param_infos, saved_views):
            setattr(self._get_module_from_name(mn), n, p)
        for (mn, n, shared_mn, shared_n) in self._shared_param_infos:
            setattr(self._get_module_from_name(mn), n, getattr(self._get_module_from_name(shared_mn), shared_n))

    @contextmanager
    def replaced_buffers(self, buffers):
        for (mn, n, _), new_b in zip(self._buffer_infos, buffers):
            setattr(self._get_module_from_name(mn), n, new_b)
        yield
        for mn, n, old_b in self._buffer_infos:
            setattr(self._get_module_from_name(mn), n, old_b)

    def _forward_with_param_and_buffers(self, flat_param, buffers, *inputs, **kwinputs):
        with self.unflattened_param(flat_param):
            with self.replaced_buffers(buffers):
                return self.module(*inputs, **kwinputs)

    def _forward_with_param(self, flat_param, *inputs, **kwinputs):
        with self.unflattened_param(flat_param):
            return self.module(*inputs, **kwinputs)

    def forward(self, *inputs, flat_param=None, buffers=None, **kwinputs):
        flat_param = torch.squeeze(flat_param)
        # print("PARAMS ON DEVICE: ", flat_param.get_device())
        # print("DATA ON DEVICE: ", inputs[0].get_device())
        # flat_param.to("cuda:{}".format(inputs[0].get_device()))
        # self.module.to("cuda:{}".format(inputs[0].get_device()))
        if flat_param is None:
            flat_param = self.flat_param
        if buffers is None:
            return self._forward_with_param(flat_param, *inputs, **kwinputs)
        else:
            return self._forward_with_param_and_buffers(flat_param, tuple(buffers), *inputs, **kwinputs)

#MNIST Data Generation

In [ ]:
args = type('', (), {})()

args.dataset = 'MNIST'

# ImageNet subset. This only does anything when --dataset=ImageNet
args.subset = 'imagenette'

# model
args.model = 'ConvNet'

# image(s) per class
args.ipc = 5

# eval_mode, check utils.py for more info
args.eval_mode = 'S'

# how many networks to evaluate on
args.num_eval = 3

# how often to evaluate
args.eval_it = 100

# epochs to train a model with synthetic data
args.epoch_eval_train = 1000

# how many distillation steps to perform
args.Iteration = 300

# Learning rates
args.lr_img = 1000  # learning rate for updating synthetic images
args.lr_teacher = 0.01  # initialization for synthetic learning rate
args.lr_init = 0.01  # how to init lr (alpha)

# Batch sizes
args.batch_real = 256  # batch size for real data
args.batch_syn = None  # should only use this if you run out of VRAM
args.batch_train = 256  # batch size for training networks

# Initialization for synthetic images
args.pix_init = 'samples_predicted_correctly'  # initialize synthetic images from random noise or real images

# Differentiable Siamese Augmentation
args.dsa = True  # whether to use differentiable Siamese augmentation
args.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'  # differentiable Siamese augmentation strategy
if args.dsa:
    # args.epoch_eval_train = 1000
    args.dc_aug_param = None

args.dsa_param = ParamDiffAug()

dsa_params = args.dsa_param


# Paths
args.data_path = './data/'  # dataset path
args.buffer_path = './buffers/'  # buffer path

# Expert epochs and synthetic data steps
args.expert_epochs = 2  # how many expert epochs the target params are
args.syn_steps = 80  # how many steps to take on synthetic data

# Start epochs
args.max_start_epoch = 5  # max epoch we can start at
args.min_start_epoch = 0  # min epoch we can start at

# ZCA whitening
args.zca = True  # do ZCA whitening (use True if action='store_true')

# Load all expert trajectories into RAM
args.load_all = False  # only use if you can fit all expert trajectories into RAM (use True if action='store_true')

# Turn off differential augmentation during distillation
args.no_aug = False  # this turns off diff aug during distillation

# Distill textures instead
args.texture = False  # will distill textures instead (use True if action='store_true')
args.canvas_size = 2  # size of synthetic canvas
args.canvas_samples = 1  # number of canvas samples per iteration

# Number of expert files to read (leave as None unless doing ablations)
args.max_files = None

# Number of experts to read per file (leave as None unless doing ablations)
args.max_experts = None

# Force saving images for 50ipc
args.force_save = False  # this will save images for 50ipc (use True if action='store_true')
args.ema_decay = 0.999


# Learning rate for 'y'
args.lr_y = 5.
# Momentum for 'y'
args.Momentum_y = 0.9

# WanDB Project Name
args.project = 'TEST'

# Threshold
args.threshold = 1.0

# Record loss
args.record_loss = False

# Sequential Generation
args.Sequential_Generation = True
args.expansion_end_epoch = 3000
args.current_max_start_epoch = 1


# Skip first evaluation
args.skip_first_eva = True  # If skip first eva

# Parallel evaluation
args.parall_eva = False  # If parallel eva

args.lr_lr = 0.00001

args.res = 32

args.device = [0]

args.Initialize_Label_With_Another_Model = False
args.Initialize_Label_Model = ""
args.Initialize_Label_Model_Dir = ""
args.Label_Model_Timestamp = -1

In [ ]:
def DATM():
  args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
  if args.skip_first_eva==False:
      eval_it_pool = np.arange(0, args.Iteration + 1, args.eval_it).tolist()
  else:
      eval_it_pool = np.arange(args.eval_it, args.Iteration + 1, args.eval_it).tolist()
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader, loader_train_dict, class_map, class_map_inv = get_dataset(args.dataset, args.data_path, args.batch_real, args.subset, args=args)
  model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)

  im_res = im_size[0]

  args.im_size = im_size

  accs_all_exps = dict() # record performances of all experiments
  for key in model_eval_pool:
      accs_all_exps[key] = []

  data_save = []

  if args.dsa:
      # args.epoch_eval_train = 1000
      args.dc_aug_param = None

  args.dsa_param = ParamDiffAug()

  dsa_params = args.dsa_param
  if args.zca:
      zca_trans = args.zca_trans
  else:
      zca_trans = None

  wandb.init(sync_tensorboard=False,
              project=args.project,
              job_type="CleanRepo",
              config=args,
              )

  #args = type('', (), {})()

  for key in wandb.config._items:
      setattr(args, key, wandb.config._items[key])

  args.dsa_param = dsa_params
  args.zca_trans = zca_trans

  if args.batch_syn is None:
      args.batch_syn = num_classes * args.ipc

  args.distributed = torch.cuda.device_count() > 1


  print('Hyper-parameters: \n', args.__dict__)
  print('Evaluation model pool: ', model_eval_pool)

  ''' organize the real dataset '''
  images_all = []
  labels_all = []
  indices_class = [[] for c in range(num_classes)]
  print("BUILDING DATASET")
  if args.dataset == 'ImageNet1K' and os.path.exists('images_all.pt') and os.path.exists('labels_all.pt'):
      images_all = torch.load('images_all.pt')
      labels_all = torch.load('labels_all.pt')
  else:
      for i in tqdm(range(len(dst_train))):
          sample = dst_train[i]
          images_all.append(torch.unsqueeze(sample[0], dim=0))
          labels_all.append(class_map[torch.tensor(sample[1]).item()])
      images_all = torch.cat(images_all, dim=0).to("cpu")
      labels_all = torch.tensor(labels_all, dtype=torch.long, device="cpu")
      if args.dataset == 'ImageNet1K':
          torch.save(images_all, 'images_all.pt')
          torch.save(labels_all, 'labels_all.pt')

  for i, lab in tqdm(enumerate(labels_all)):
      indices_class[lab].append(i)



  for c in range(num_classes):
      print('class c = %d: %d real images'%(c, len(indices_class[c])))

  for ch in range(channel):
      print('real images channel %d, mean = %.4f, std = %.4f'%(ch, torch.mean(images_all[:, ch]), torch.std(images_all[:, ch])))


  def get_images(c, n):  # get random n images from class c
      idx_shuffle = np.random.permutation(indices_class[c])[:n]
      return images_all[idx_shuffle]
  ''' initialize the synthetic data '''
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


  image_syn = torch.randn(size=(num_classes * args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float)

  syn_lr = torch.tensor(args.lr_teacher).to(args.device)
  expert_dir = os.path.join(args.buffer_path, args.dataset)
  if args.dataset == "ImageNet":
    expert_dir = os.path.join(expert_dir, args.subset, str(args.res))
  if args.dataset in ["CIFAR10", "CIFAR100"] and not args.zca:
    expert_dir += "_NO_ZCA"
  expert_dir = os.path.join(expert_dir, args.model)
  print("Expert Dir: {}".format(expert_dir))
  if args.load_all:
    buffer = []
    n = 0
    while os.path.exists(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n))):
        buffer = buffer + torch.load(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n)))
        n += 1
    if n == 0:
        raise AssertionError("No buffers detected at {}".format(expert_dir))

  else:
    expert_files = []
    n = 0
    while os.path.exists(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n))):
        expert_files.append(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n)))
        n += 1
    if n == 0:
        raise AssertionError("No buffers detected at {}".format(expert_dir))
    file_idx = 0
    expert_idx = 0
    # random.shuffle(expert_files)
    if args.max_files is not None:
        expert_files = expert_files[:args.max_files]

    expert_id = [i for i in range(len(expert_files))]
    random.shuffle(expert_id)

    print("loading file {}".format(expert_files[expert_id[file_idx]]))
    buffer = torch.load(expert_files[expert_id[file_idx]])
    if args.max_experts is not None:
        buffer = buffer[:args.max_experts]
    buffer_id = [i for i in range(len(buffer))]
    random.shuffle(buffer_id)

    if args.pix_init == 'real':
        print('initialize synthetic data from random real images')
        for c in range(num_classes):
            image_syn.data[c * args.ipc:(c + 1) * args.ipc] = get_images(c, args.ipc).detach().data

    elif args.pix_init == 'samples_predicted_correctly':
        if args.parall_eva==False:
            device = torch.device("cuda:0")
        else:
            device = args.device
        if args.Initialize_Label_With_Another_Model:
            Temp_net = get_network(args.Initialize_Label_Model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
        else:
            Temp_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
        Temp_net.eval()
        Temp_net = ReparamModule(Temp_net)
        if args.distributed and args.parall_eva==True:
            Temp_net = torch.nn.DataParallel(Temp_net)
        Temp_net.eval()
        logits=[]
        label_expert_files = expert_files
        temp_params = torch.load(label_expert_files[0])[0][args.Label_Model_Timestamp]
        temp_params = torch.cat([p.data.to(device).reshape(-1) for p in temp_params], 0)
        if args.distributed and args.parall_eva==True:
            temp_params = temp_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
        for c in range(num_classes):
            data_for_class_c = get_images(c, len(indices_class[c])).detach().data
            n, _, w, h = data_for_class_c.shape
            selected_num = 0
            select_times = 0
            cur=0
            temp_img = None
            Wrong_Predicted_Img = None
            batch_size = 256
            index = []
            while len(index)<args.ipc:
                print(str(c)+'.'+str(select_times)+'.'+str(cur))
                current_data_batch = data_for_class_c[batch_size*select_times : batch_size*(select_times+1)].detach().to(device)
                if batch_size*select_times > len(data_for_class_c):
                    select_times = 0
                    cur+=1
                    temp_params = torch.load(label_expert_files[int(cur/10)%10])[cur%10][args.Label_Model_Timestamp]
                    temp_params = torch.cat([p.data.to(device).reshape(-1) for p in temp_params], 0).to(device)
                    if args.distributed and args.parall_eva==True:
                        temp_params = temp_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
                    continue
                logits = Temp_net(current_data_batch, flat_param=temp_params).detach()
                prediction_class = np.argmax(logits.cpu().data.numpy(), axis=-1)
                for i in range(len(prediction_class)):
                    if prediction_class[i]==c and len(index)<args.ipc:
                        index.append(batch_size*select_times+i)
                        index=list(set(index))
                select_times+=1
                if len(index) == args.ipc:
                    temp_img = torch.index_select(data_for_class_c, dim=0, index=torch.tensor(index))
                    break
            image_syn.data[c * args.ipc:(c + 1) * args.ipc] = temp_img.detach()
    else:
        print('initialize synthetic data from random noise')

    ''' training '''
    image_syn = image_syn.detach().to(args.device).requires_grad_(True)
    syn_lr = syn_lr.detach().to(args.device).requires_grad_(True)

    #optimizer_img = torch.optim.SGD([image_syn], lr=args.lr_img, momentum=0.1)
    #optimizer_lr = torch.optim.SGD([syn_lr], lr=args.lr_lr, momentum=0.1)

    optimizer_img = torch.optim.SGD([image_syn], lr=args.lr_img, momentum=0.5)
    optimizer_lr = torch.optim.SGD([syn_lr], lr=args.lr_lr, momentum=0.5)





    optimizer_img.zero_grad()

    ###

    '''test'''
    def SoftCrossEntropy(inputs, target, reduction='average'):
        input_log_likelihood = -F.log_softmax(inputs, dim=1)
        target_log_likelihood = F.softmax(target, dim=1)
        batch = inputs.shape[0]
        loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
        return loss

    criterion = SoftCrossEntropy

    print('%s training begins'%get_time())
    best_acc = {m: 0 for m in model_eval_pool}
    best_std = {m: 0 for m in model_eval_pool}

    '''------test------'''
    '''only sum correct predicted logits'''
    if args.pix_init == "samples_predicted_correctly":
        if args.Initialize_Label_With_Another_Model:
            Temp_net = get_network(args.Initialize_Label_Model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
        else:
            Temp_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
        Temp_net.eval()
        Temp_net = ReparamModule(Temp_net)
        if args.distributed:
            Temp_net = torch.nn.DataParallel(Temp_net)
        Temp_net.eval()
        logits=[]
        batch_size = 256
        for i in range(len(label_expert_files)):
            Temp_Buffer = torch.load(label_expert_files[i])
            for j in Temp_Buffer:
                temp_logits = None
                for select_times in range((len(image_syn)+batch_size-1)//batch_size):
                    current_data_batch = image_syn[batch_size*select_times : batch_size*(select_times+1)].detach().to(device)
                    Temp_params = j[args.Label_Model_Timestamp]
                    Initialize_Labels_params = torch.cat([p.data.to(args.device).reshape(-1) for p in Temp_params], 0)
                    if args.distributed:
                        Initialize_Labels_params = Initialize_Labels_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
                    Initialized_Labels = Temp_net(current_data_batch, flat_param=Initialize_Labels_params)
                    if temp_logits == None:
                        temp_logits = Initialized_Labels.detach()
                    else:
                        temp_logits = torch.cat((temp_logits, Initialized_Labels.detach()),0)
                logits.append(temp_logits.detach().cpu())
        logits_tensor = torch.stack(logits)
        true_labels = label_syn.cpu()
        predicted_labels = torch.argmax(logits_tensor, dim=2).cpu()
        correct_predictions = predicted_labels == true_labels.view(1, -1)
        mask = correct_predictions.unsqueeze(2)
        correct_logits = logits_tensor * mask.float()
        correct_logits_per_model = correct_logits.sum(dim=0)
        num_correct_images_per_model = correct_predictions.sum(dim=0, dtype=torch.float)
        average_logits_per_image = correct_logits_per_model / num_correct_images_per_model.unsqueeze(1)
        Initialized_Labels = average_logits_per_image

    elif args.pix_init == "real":
        Temp_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(args.device)  # get a random model
        Temp_net = ReparamModule(Temp_net)
        if args.distributed:
            Temp_net = torch.nn.DataParallel(Temp_net)
        Temp_net.eval()
        Temp_params = buffer[0][-1]
        Initialize_Labels_params = torch.cat([p.data.to(args.device).reshape(-1) for p in Temp_params], 0)
        if args.distributed:
            Initialize_Labels_params = Initialize_Labels_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
        Initialized_Labels = Temp_net(image_syn, flat_param=Initialize_Labels_params)

    acc = np.sum(np.equal(np.argmax(Initialized_Labels.cpu().data.numpy(), axis=-1), label_syn.cpu().data.numpy()))
    print('InitialAcc:{}'.format(acc/len(label_syn)))

    label_syn = copy.deepcopy(Initialized_Labels.detach()).to(args.device).requires_grad_(True)
    label_syn.requires_grad=True
    label_syn = label_syn.to(args.device)


    optimizer_y = torch.optim.SGD([label_syn], lr=args.lr_y, momentum=args.Momentum_y)
    vs = torch.zeros_like(label_syn)
    accumulated_grad = torch.zeros_like(label_syn)
    last_random = 0

    del Temp_net

    # test
    curMax_times = 0
    current_accumulated_step = 0

    for it in range(0, args.Iteration+1):
        save_this_it = False
        wandb.log({"Progress": it}, step=it)
        ''' Evaluate synthetic data '''
        if it in eval_it_pool:
            for model_eval in model_eval_pool:
                print('-------------------------\nEvaluation\nmodel_train = %s, model_eval = %s, iteration = %d'%(args.model, model_eval, it))
                if args.dsa:
                    print('DSA augmentation strategy: \n', args.dsa_strategy)
                    print('DSA augmentation parameters: \n', args.dsa_param.__dict__)
                else:
                    print('DC augmentation parameters: \n', args.dc_aug_param)

                accs_test = []
                accs_train = []

                for it_eval in range(args.num_eval):
                    if args.parall_eva==False:
                        device = torch.device("cuda:0")
                        net_eval = get_network(model_eval, channel, num_classes, im_size, dist=False).to(device) # get a random model
                    else:
                        device = args.device
                        net_eval = get_network(model_eval, channel, num_classes, im_size, dist=True).to(device) # get a random model

                    eval_labs = label_syn.detach().to(device)
                    with torch.no_grad():
                        image_save = image_syn.to(device)
                    image_syn_eval, label_syn_eval = copy.deepcopy(image_save.detach()).to(device), copy.deepcopy(eval_labs.detach()).to(device) # avoid any unaware modification

                    args.lr_net = syn_lr.item()
                    _, acc_train, acc_test = evaluate_synset(it_eval, copy.deepcopy(net_eval).to(device), image_syn_eval.to(device), label_syn_eval.to(device), testloader, args, texture=False, train_criterion=criterion)
                    accs_test.append(acc_test)
                    accs_train.append(acc_train)

                accs_test = np.array(accs_test)
                accs_train = np.array(accs_train)
                acc_test_mean = np.mean(accs_test)
                acc_test_std = np.std(accs_test)

                if acc_test_mean > best_acc[model_eval]:
                    best_acc[model_eval] = acc_test_mean
                    best_std[model_eval] = acc_test_std
                    save_this_it = True
                print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs_test), model_eval, acc_test_mean, acc_test_std))
                wandb.log({'Accuracy/{}'.format(model_eval): acc_test_mean}, step=it)
                wandb.log({'Max_Accuracy/{}'.format(model_eval): best_acc[model_eval]}, step=it)
                wandb.log({'Std/{}'.format(model_eval): acc_test_std}, step=it)
                wandb.log({'Max_Std/{}'.format(model_eval): best_std[model_eval]}, step=it)

        if it in eval_it_pool and (save_this_it or it % 1000 == 0):
            with torch.no_grad():
                image_save = image_syn.cuda()
                save_dir = os.path.join(".", "logged_files", args.dataset, str(args.ipc), args.model, wandb.run.name)

                if not os.path.exists(save_dir):
                    os.makedirs(os.path.join(save_dir,'Normal'))

                torch.save(image_save.cpu(), os.path.join(save_dir, 'Normal',"images_{}.pt".format(it)))
                torch.save(label_syn.cpu(), os.path.join(save_dir, 'Normal', "labels_{}.pt".format(it)))
                torch.save(syn_lr.detach().cpu(), os.path.join(save_dir, 'Normal', "lr_{}.pt".format(it)))

                if save_this_it:
                    torch.save(image_save.cpu(), os.path.join(save_dir, 'Normal', "images_best.pt".format(it)))
                    torch.save(label_syn.cpu(), os.path.join(save_dir, 'Normal', "labels_best.pt".format(it)))
                    torch.save(syn_lr.detach().cpu(), os.path.join(save_dir, 'Normal', "lr_best.pt".format(it)))

                wandb.log({"Pixels": wandb.Histogram(torch.nan_to_num(image_syn.detach().cpu()))}, step=it)

                if args.ipc < 50 or args.force_save:
                    upsampled = image_save
                    if args.dataset != "ImageNet":
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                    grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                    wandb.log({"Synthetic_Images": wandb.Image(torch.nan_to_num(grid.detach().cpu()))}, step=it)
                    wandb.log({'Synthetic_Pixels': wandb.Histogram(torch.nan_to_num(image_save.detach().cpu()))}, step=it)

                    for clip_val in [2.5]:
                        std = torch.std(image_save)
                        mean = torch.mean(image_save)
                        upsampled = torch.clip(image_save, min=mean-clip_val*std, max=mean+clip_val*std)
                        if args.dataset != "ImageNet":
                            upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                            upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                        grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                        wandb.log({"Clipped_Synthetic_Images/std_{}".format(clip_val): wandb.Image(torch.nan_to_num(grid.detach().cpu()))}, step=it)

                    if args.zca:
                        image_save = image_save.to(args.device)
                        image_save = args.zca_trans.inverse_transform(image_save)
                        image_save.cpu()
                        torch.save(image_save.cpu(), os.path.join(save_dir, 'Normal', "images_zca_{}.pt".format(it)))
                        upsampled = image_save
                        if args.dataset != "ImageNet":
                            upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                            upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                        grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                        wandb.log({"Reconstructed_Images": wandb.Image(torch.nan_to_num(grid.detach().cpu()))}, step=it)
                        wandb.log({'Reconstructed_Pixels': wandb.Histogram(torch.nan_to_num(image_save.detach().cpu()))}, step=it)
                        for clip_val in [2.5]:
                            std = torch.std(image_save)
                            mean = torch.mean(image_save)
                            upsampled = torch.clip(image_save, min=mean - clip_val * std, max=mean + clip_val * std)
                            if args.dataset != "ImageNet":
                                upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                                upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                            grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                            wandb.log({"Clipped_Reconstructed_Images/std_{}".format(clip_val): wandb.Image(
                                torch.nan_to_num(grid.detach().cpu()))}, step=it)



        wandb.log({"Synthetic_LR": syn_lr.detach().cpu()}, step=it)

        student_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(args.device)  # get a random model

        student_net = ReparamModule(student_net)

        if args.distributed:
            student_net = torch.nn.DataParallel(student_net)

        student_net.train()

        num_params = sum([np.prod(p.size()) for p in (student_net.parameters())])

        if args.load_all:
            expert_trajectory = buffer[np.random.randint(0, len(buffer))]
        else:
            expert_trajectory = buffer[buffer_id[expert_idx]]
            expert_idx += 1
            if expert_idx == len(buffer):
                expert_idx = 0
                file_idx += 1
                if file_idx == len(expert_files):
                    file_idx = 0
                    random.shuffle(expert_id)
                print("loading file {}".format(expert_files[expert_id[file_idx]]))
                if args.max_files != 1:
                    del buffer
                    buffer = torch.load(expert_files[expert_id[file_idx]])
                if args.max_experts is not None:
                    buffer = buffer[:args.max_experts]
                random.shuffle(buffer_id)

        # Only match easy traj. in the early stage
        if args.Sequential_Generation:
            Upper_Bound = args.current_max_start_epoch + int((args.max_start_epoch-args.current_max_start_epoch) * it/(args.expansion_end_epoch))
            Upper_Bound = min(Upper_Bound, args.max_start_epoch)
        else:
            Upper_Bound = args.max_start_epoch

        start_epoch = np.random.randint(args.min_start_epoch, Upper_Bound)

        starting_params = expert_trajectory[start_epoch]
        target_params = expert_trajectory[start_epoch+args.expert_epochs]
        target_params = torch.cat([p.data.to(args.device).reshape(-1) for p in target_params], 0)
        student_params = [torch.cat([p.data.to(args.device).reshape(-1) for p in starting_params], 0).requires_grad_(True)]
        starting_params = torch.cat([p.data.to(args.device).reshape(-1) for p in starting_params], 0)

        syn_images = image_syn
        y_hat = label_syn

        param_loss_list = []
        param_dist_list = []
        indices_chunks = []




        for step in range(args.syn_steps):
            if not indices_chunks:
                indices = torch.randperm(len(syn_images))
                indices_chunks = list(torch.split(indices, args.batch_syn))

            these_indices = indices_chunks.pop()

            x = syn_images[these_indices]
            this_y = y_hat[these_indices]


            if args.dsa and (not args.no_aug):
                x = DiffAugment(x, args.dsa_strategy, param=args.dsa_param)

            if args.distributed:
                forward_params = student_params[-1].unsqueeze(0).expand(torch.cuda.device_count(), -1)
            else:
                forward_params = student_params[-1]
            x = student_net(x, flat_param=forward_params)
            ce_loss = criterion(x, this_y)

            grad = torch.autograd.grad(ce_loss, student_params[-1], create_graph=True)[0]

            student_params.append(student_params[-1] - syn_lr * grad)

        param_loss = torch.tensor(0.0).to(args.device)
        param_dist = torch.tensor(0.0).to(args.device)

        param_loss += torch.nn.functional.mse_loss(student_params[-1], target_params, reduction="sum")
        param_dist += torch.nn.functional.mse_loss(starting_params, target_params, reduction="sum")

        param_loss_list.append(param_loss)
        param_dist_list.append(param_dist)

        param_loss /= num_params
        param_dist /= num_params

        param_loss /= param_dist

        grand_loss = param_loss

        optimizer_img.zero_grad()
        optimizer_lr.zero_grad()
        optimizer_y.zero_grad()

        grand_loss.backward()

        if grand_loss<=args.threshold:
            optimizer_y.step()
            optimizer_img.step()
            optimizer_lr.step()
        else:
            wandb.log({"falts": start_epoch}, step=it)



        wandb.log({"Grand_Loss": param_loss.detach().cpu(),
                    "Start_Epoch": start_epoch})

        for _ in student_params:
            del _

        if it%10 == 0:
            print('%s iter = %04d, loss = %.4f' % (get_time(), it, grand_loss.item()))


    wandb.finish()








In [ ]:
DATM()

Train ZCA


100%|██████████| 60000/60000 [00:16<00:00, 3663.71it/s]


Test ZCA


100%|██████████| 10000/10000 [00:02<00:00, 4191.45it/s]


<class 'kornia.enhance.zca.ZCAWhitening'>


...

 99%|█████████▉| 992/1001 [00:09<00:00, 117.92it/s]

[2023-11-27 20:49:32] Evaluate_02: epoch = 0977 train loss = 0.100212 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0978 train loss = 0.087809 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0979 train loss = 0.088684 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0980 train loss = 0.088934 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0981 train loss = 0.114206 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0982 train loss = 0.089515 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0983 train loss = 0.099248 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0984 train loss = 0.108177 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0985 train loss = 0.088779 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0986 train loss = 0.087062 train acc = 0.0200
[2023-11-27 20:49:32] Evaluate_02: epoch = 0987 train loss = 0.099899 train acc = 0.0200
[2023-11-27 20:49:32]

100%|██████████| 1001/1001 [00:09<00:00, 101.20it/s]


[2023-11-27 20:49:33] Evaluate_02: epoch = 1000 train time = 9 s train loss = 0.089163 train acc = 0.0200, test acc = 0.9140
Evaluate 3 random ConvNet, mean = 0.9139 std = 0.0007
-------------------------
loading file ./buffers/MNIST/ConvNet/replay_buffer_19.pt
[2023-11-27 20:49:36] iter = 0300, loss = 0.2192


Accuracy/ConvNet,▁▆█
Grand_Loss,█▇▅▅▄▄▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▂▁▁▁▁▂▁▁▁▂▁
Max_Accuracy/ConvNet,▁▆█
Max_Std/ConvNet,█▇▁
Progress,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Start_Epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Std/ConvNet,█▇▁
Synthetic_LR,▁▁▂▂▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████
Accuracy/ConvNet,0.91387
Grand_Loss,0.21923
Max_Accuracy/ConvNet,0.91387


#MHIST Data Generation

In [ ]:
args = type('', (), {})()

args.dataset = 'MHIST'

# ImageNet subset. This only does anything when --dataset=ImageNet
args.subset = 'imagenette'

# model
args.model = 'ConvNetD5'

# image(s) per class
args.ipc = 100

# eval_mode, check utils.py for more info
args.eval_mode = 'S'

# how many networks to evaluate on
args.num_eval = 1

# how often to evaluate
args.eval_it = 200

# epochs to train a model with synthetic data
args.epoch_eval_train = 1000

# how many distillation steps to perform
args.Iteration = 1000

# Learning rates
args.lr_img = 10  # learning rate for updating synthetic images
args.lr_teacher = 0.01  # initialization for synthetic learning rate
args.lr_init = 0.01  # how to init lr (alpha)

# Batch sizes
args.batch_real = 32  # batch size for real data
args.batch_syn = 100  # should only use this if you run out of VRAM
args.batch_train = 32  # batch size for training networks

# Initialization for synthetic images
args.pix_init = 'samples_predicted_correctly'  # initialize synthetic images from random noise or real images

# Differentiable Siamese Augmentation
args.dsa = True  # whether to use differentiable Siamese augmentation
args.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'  # differentiable Siamese augmentation strategy
if args.dsa:
    # args.epoch_eval_train = 1000
    args.dc_aug_param = None

args.dsa_param = ParamDiffAug()

dsa_params = args.dsa_param


# Paths
args.data_path = './data/'  # dataset path
args.buffer_path = './buffers/'  # buffer path

# Expert epochs and synthetic data steps
args.expert_epochs = 2  # how many expert epochs the target params are
args.syn_steps = 40  # how many steps to take on synthetic data

# Start epochs
args.max_start_epoch = 35  # max epoch we can start at
args.min_start_epoch = 5  # min epoch we can start at

# ZCA whitening
args.zca = True  # do ZCA whitening (use True if action='store_true')

# Load all expert trajectories into RAM
args.load_all = False  # only use if you can fit all expert trajectories into RAM (use True if action='store_true')

# Turn off differential augmentation during distillation
args.no_aug = False  # this turns off diff aug during distillation

# Distill textures instead
args.texture = False  # will distill textures instead (use True if action='store_true')
args.canvas_size = 2  # size of synthetic canvas
args.canvas_samples = 1  # number of canvas samples per iteration

# Number of expert files to read (leave as None unless doing ablations)
args.max_files = None

# Number of experts to read per file (leave as None unless doing ablations)
args.max_experts = None

# Force saving images for 50ipc
args.force_save = True  # this will save images for 50ipc (use True if action='store_true')
args.ema_decay = 0.999


# Learning rate for 'y'
args.lr_y = 10.
# Momentum for 'y'
args.Momentum_y = 0.9

# WanDB Project Name
args.project = 'TEST'

# Threshold
args.threshold = 1.0

# Record loss
args.record_loss = False

# Sequential Generation
args.Sequential_Generation = False
args.expansion_end_epoch = 3000
args.current_max_start_epoch = 10


# Skip first evaluation
args.skip_first_eva = False  # If skip first eva

# Parallel evaluation
args.parall_eva = False  # If parallel eva

args.lr_lr = 0.0001

args.res = 32

args.device = [0]

args.Initialize_Label_With_Another_Model = False
args.Initialize_Label_Model = ""
args.Initialize_Label_Model_Dir = ""
args.Label_Model_Timestamp = -1

In [ ]:
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
if args.skip_first_eva==False:
    eval_it_pool = np.arange(0, args.Iteration + 1, args.eval_it).tolist()
else:
    eval_it_pool = np.arange(args.eval_it, args.Iteration + 1, args.eval_it).tolist()
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader, loader_train_dict, class_map, class_map_inv = get_dataset(args.dataset, args.data_path, args.batch_real, args.subset, args=args)
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)

im_res = im_size[0]

args.im_size = im_size

accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []

data_save = []

if args.dsa:
    # args.epoch_eval_train = 1000
    args.dc_aug_param = None

args.dsa_param = ParamDiffAug()

dsa_params = args.dsa_param
if args.zca:
    zca_trans = args.zca_trans
else:
    zca_trans = None

wandb.init(sync_tensorboard=False,
            project=args.project,
            job_type="CleanRepo",
            config=args,
            )

#args = type('', (), {})()

for key in wandb.config._items:
    setattr(args, key, wandb.config._items[key])

args.dsa_param = dsa_params
args.zca_trans = zca_trans

if args.batch_syn is None:
    args.batch_syn = num_classes * args.ipc

args.distributed = torch.cuda.device_count() > 1


print('Hyper-parameters: \n', args.__dict__)
print('Evaluation model pool: ', model_eval_pool)

''' organize the real dataset '''
images_all = []
labels_all = []
indices_class = [[] for c in range(num_classes)]
print("BUILDING DATASET")
if args.dataset == 'ImageNet1K' and os.path.exists('images_all.pt') and os.path.exists('labels_all.pt'):
    images_all = torch.load('images_all.pt')
    labels_all = torch.load('labels_all.pt')
else:
    for i in tqdm(range(len(dst_train))):
        sample = dst_train[i]
        images_all.append(torch.unsqueeze(sample[0], dim=0))
        labels_all.append(class_map[torch.tensor(sample[1]).item()])
    images_all = torch.cat(images_all, dim=0).to("cpu")
    labels_all = torch.tensor(labels_all, dtype=torch.long, device="cpu")
    if args.dataset == 'ImageNet1K':
        torch.save(images_all, 'images_all.pt')
        torch.save(labels_all, 'labels_all.pt')

for i, lab in tqdm(enumerate(labels_all)):
    indices_class[lab].append(i)



for c in range(num_classes):
    print('class c = %d: %d real images'%(c, len(indices_class[c])))

for ch in range(channel):
    print('real images channel %d, mean = %.4f, std = %.4f'%(ch, torch.mean(images_all[:, ch]), torch.std(images_all[:, ch])))


def get_images(c, n):  # get random n images from class c
    idx_shuffle = np.random.permutation(indices_class[c])[:n]
    return images_all[idx_shuffle]


Train ZCA


100%|██████████| 6304/6304 [00:41<00:00, 152.51it/s]


Test ZCA


100%|██████████| 977/977 [03:13<00:00,  5.06it/s]


<class 'kornia.enhance.zca.ZCAWhitening'>


...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Hyper-parameters: 
 {'dataset': 'MHIST', 'subset': 'imagenette', 'model': 'ConvNetD5', 'ipc': 100, 'eval_mode': 'S', 'num_eval': 1, 'eval_it': 200, 'epoch_eval_train': 1000, 'Iteration': 1000, 'lr_img': 10, 'lr_teacher': 0.01, 'lr_init': 0.01, 'batch_real': 32, 'batch_syn': 100, 'batch_train': 32, 'pix_init': 'samples_predicted_correctly', 'dsa': True, 'dsa_strategy': 'color_crop_cutout_flip_scale_rotate', 'dc_aug_param': None, 'dsa_param': <__main__.ParamDiffAug object at 0x7fcabe070280>, 'data_path': './data/', 'buffer_path': './buffers/', 'expert_epochs': 2, 'syn_steps': 40, 'max_start_epoch': 35, 'min_start_epoch': 5, 'zca': True, 'load_all': False, 'no_aug': False, 'texture': False, 'canvas_size': 2, 'canvas_samples': 1, 'max_files': None, 'max_experts': None, 'force_save': True, 'ema_decay': 0.999, 'lr_y': 10.0, 'Momentum_y': 0.9, 'project': 'TEST', 'threshold': 1.0, 'record_loss': False, 'Sequential_Generation': False, 'expansion_end_epoch': 3000, 'current_max_start_epoch': 10, 

  0%|          | 0/6304 [00:00<?, ?it/s]<ipython-input-47-83cb6ff5197f>:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_all.append(class_map[torch.tensor(sample[1]).item()])
100%|██████████| 6304/6304 [00:00<00:00, 15945.90it/s]
6304it [00:00, 374676.10it/s]


class c = 0: 4324 real images
class c = 1: 1980 real images
real images channel 0, mean = -0.0000, std = 0.2058
real images channel 1, mean = 0.0000, std = 0.2637
real images channel 2, mean = -0.0000, std = 0.1671


In [ ]:
''' initialize the synthetic data '''
label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


image_syn = torch.randn(size=(num_classes * args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float)

syn_lr = torch.tensor(args.lr_teacher).to(args.device)
expert_dir = os.path.join(args.buffer_path, args.dataset)
if args.dataset == "ImageNet":
  expert_dir = os.path.join(expert_dir, args.subset, str(args.res))
if args.dataset in ["CIFAR10", "CIFAR100"] and not args.zca:
  expert_dir += "_NO_ZCA"
expert_dir = os.path.join(expert_dir, args.model)
print("Expert Dir: {}".format(expert_dir))
if args.load_all:
  buffer = []
  n = 0
  while os.path.exists(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n))):
      buffer = buffer + torch.load(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n)))
      n += 1
  if n == 0:
      raise AssertionError("No buffers detected at {}".format(expert_dir))

else:
  expert_files = []
  n = 0
  while os.path.exists(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n))):
      expert_files.append(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n)))
      n += 1
  if n == 0:
      raise AssertionError("No buffers detected at {}".format(expert_dir))
  file_idx = 0
  expert_idx = 0
  # random.shuffle(expert_files)
  if args.max_files is not None:
      expert_files = expert_files[:args.max_files]

  expert_id = [i for i in range(len(expert_files))]
  random.shuffle(expert_id)

  print("loading file {}".format(expert_files[expert_id[file_idx]]))
  buffer = torch.load(expert_files[expert_id[file_idx]])
  if args.max_experts is not None:
      buffer = buffer[:args.max_experts]
  buffer_id = [i for i in range(len(buffer))]
  random.shuffle(buffer_id)

<ipython-input-24-ad36cd613f1f>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


Expert Dir: ./buffers/MHIST/ConvNetD5
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_6.pt


In [ ]:
len(buffer[buffer_id[expert_idx]])

51

In [ ]:
#expert_trajectory = buffer[buffer_id[expert_idx]]
len(buffer)
expert_idx
buffer_id

[0]

In [ ]:
if args.pix_init == 'real':
    print('initialize synthetic data from random real images')
    for c in range(num_classes):
        image_syn.data[c * args.ipc:(c + 1) * args.ipc] = get_images(c, args.ipc).detach().data


elif args.pix_init == 'samples_predicted_correctly':
    if args.parall_eva==False:
        device = torch.device("cuda:0")
    else:
        device = args.device
    if args.Initialize_Label_With_Another_Model:
        Temp_net = get_network(args.Initialize_Label_Model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
    else:
        Temp_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
    Temp_net.eval()
    Temp_net = ReparamModule(Temp_net)
    if args.distributed and args.parall_eva==True:
        Temp_net = torch.nn.DataParallel(Temp_net)
    Temp_net.eval()
    logits=[]
    label_expert_files = expert_files
    temp_params = torch.load(label_expert_files[0])[0][args.Label_Model_Timestamp]
    temp_params = torch.cat([p.data.to(device).reshape(-1) for p in temp_params], 0)
    if args.distributed and args.parall_eva==True:
        temp_params = temp_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
    for c in range(num_classes):
        data_for_class_c = get_images(c, len(indices_class[c])).detach().data
        n, _, w, h = data_for_class_c.shape
        selected_num = 0
        select_times = 0
        cur=0
        temp_img = None
        Wrong_Predicted_Img = None
        batch_size = 256
        index = []
        while len(index)<args.ipc:
            print(str(c)+'.'+str(select_times)+'.'+str(cur))
            current_data_batch = data_for_class_c[batch_size*select_times : batch_size*(select_times+1)].detach().to(device)
            if batch_size*select_times > len(data_for_class_c):
                select_times = 0
                cur+=1
                temp_params = torch.load(label_expert_files[int(cur/10)%10])[cur%10][args.Label_Model_Timestamp]
                temp_params = torch.cat([p.data.to(device).reshape(-1) for p in temp_params], 0).to(device)
                if args.distributed and args.parall_eva==True:
                    temp_params = temp_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
                continue
            logits = Temp_net(current_data_batch, flat_param=temp_params).detach()
            prediction_class = np.argmax(logits.cpu().data.numpy(), axis=-1)
            for i in range(len(prediction_class)):
                if prediction_class[i]==c and len(index)<args.ipc:
                    index.append(batch_size*select_times+i)
                    index=list(set(index))
            select_times+=1
            if len(index) == args.ipc:
                temp_img = torch.index_select(data_for_class_c, dim=0, index=torch.tensor(index))
                break
        image_syn.data[c * args.ipc:(c + 1) * args.ipc] = temp_img.detach()
else:
    print('initialize synthetic data from random noise')


0.0.0
1.0.0


In [ ]:
''' training '''
image_syn = image_syn.detach().to(args.device).requires_grad_(True)
syn_lr = syn_lr.detach().to(args.device).requires_grad_(True)

#optimizer_img = torch.optim.SGD([image_syn], lr=args.lr_img, momentum=0.1)
#optimizer_lr = torch.optim.SGD([syn_lr], lr=args.lr_lr, momentum=0.1)

optimizer_img = torch.optim.SGD([image_syn], lr=args.lr_img, momentum=0.5)
optimizer_lr = torch.optim.SGD([syn_lr], lr=args.lr_lr, momentum=0.5)





optimizer_img.zero_grad()

###

'''test'''
def SoftCrossEntropy(inputs, target, reduction='average'):
    input_log_likelihood = -F.log_softmax(inputs, dim=1)
    target_log_likelihood = F.softmax(target, dim=1)
    batch = inputs.shape[0]
    loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
    return loss

criterion = SoftCrossEntropy

print('%s training begins'%get_time())
best_acc = {m: 0 for m in model_eval_pool}
best_std = {m: 0 for m in model_eval_pool}

'''------test------'''
'''only sum correct predicted logits'''
if args.pix_init == "samples_predicted_correctly":
    if args.Initialize_Label_With_Another_Model:
        Temp_net = get_network(args.Initialize_Label_Model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
    else:
        Temp_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(device)  # get a random model
    Temp_net.eval()
    Temp_net = ReparamModule(Temp_net)
    if args.distributed:
        Temp_net = torch.nn.DataParallel(Temp_net)
    Temp_net.eval()
    logits=[]
    batch_size = 256
    for i in range(len(label_expert_files)):
        Temp_Buffer = torch.load(label_expert_files[i])
        for j in Temp_Buffer:
            temp_logits = None
            for select_times in range((len(image_syn)+batch_size-1)//batch_size):
                current_data_batch = image_syn[batch_size*select_times : batch_size*(select_times+1)].detach().to(device)
                Temp_params = j[args.Label_Model_Timestamp]
                Initialize_Labels_params = torch.cat([p.data.to(args.device).reshape(-1) for p in Temp_params], 0)
                if args.distributed:
                    Initialize_Labels_params = Initialize_Labels_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
                Initialized_Labels = Temp_net(current_data_batch, flat_param=Initialize_Labels_params)
                if temp_logits == None:
                    temp_logits = Initialized_Labels.detach()
                else:
                    temp_logits = torch.cat((temp_logits, Initialized_Labels.detach()),0)
            logits.append(temp_logits.detach().cpu())
    logits_tensor = torch.stack(logits)
    true_labels = label_syn.cpu()
    predicted_labels = torch.argmax(logits_tensor, dim=2).cpu()
    correct_predictions = predicted_labels == true_labels.view(1, -1)
    mask = correct_predictions.unsqueeze(2)
    correct_logits = logits_tensor * mask.float()
    correct_logits_per_model = correct_logits.sum(dim=0)
    num_correct_images_per_model = correct_predictions.sum(dim=0, dtype=torch.float)
    average_logits_per_image = correct_logits_per_model / num_correct_images_per_model.unsqueeze(1)
    Initialized_Labels = average_logits_per_image

elif args.pix_init == "real":
    Temp_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(args.device)  # get a random model
    Temp_net = ReparamModule(Temp_net)
    if args.distributed:
        Temp_net = torch.nn.DataParallel(Temp_net)
    Temp_net.eval()
    Temp_params = buffer[0][-1]
    Initialize_Labels_params = torch.cat([p.data.to(args.device).reshape(-1) for p in Temp_params], 0)
    if args.distributed:
        Initialize_Labels_params = Initialize_Labels_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
    Initialized_Labels = Temp_net(image_syn, flat_param=Initialize_Labels_params)

acc = np.sum(np.equal(np.argmax(Initialized_Labels.cpu().data.numpy(), axis=-1), label_syn.cpu().data.numpy()))
print('InitialAcc:{}'.format(acc/len(label_syn)))

label_syn = copy.deepcopy(Initialized_Labels.detach()).to(args.device).requires_grad_(True)
label_syn.requires_grad=True
label_syn = label_syn.to(args.device)


optimizer_y = torch.optim.SGD([label_syn], lr=args.lr_y, momentum=args.Momentum_y)
vs = torch.zeros_like(label_syn)
accumulated_grad = torch.zeros_like(label_syn)
last_random = 0

del Temp_net

[2023-11-27 15:45:18] training begins
InitialAcc:1.0


In [ ]:
# test
curMax_times = 0
current_accumulated_step = 0

for it in range(0, args.Iteration+1):
    save_this_it = False
    wandb.log({"Progress": it}, step=it)
    ''' Evaluate synthetic data '''
    if it in eval_it_pool:
        for model_eval in model_eval_pool:
            print('-------------------------\nEvaluation\nmodel_train = %s, model_eval = %s, iteration = %d'%(args.model, model_eval, it))
            if args.dsa:
                print('DSA augmentation strategy: \n', args.dsa_strategy)
                print('DSA augmentation parameters: \n', args.dsa_param.__dict__)
            else:
                print('DC augmentation parameters: \n', args.dc_aug_param)

            accs_test = []
            accs_train = []

            for it_eval in range(args.num_eval):
                if args.parall_eva==False:
                    device = torch.device("cuda:0")
                    net_eval = get_network(model_eval, channel, num_classes, im_size, dist=False).to(device) # get a random model
                else:
                    device = args.device
                    net_eval = get_network(model_eval, channel, num_classes, im_size, dist=True).to(device) # get a random model

                eval_labs = label_syn.detach().to(device)
                with torch.no_grad():
                    image_save = image_syn.to(device)
                image_syn_eval, label_syn_eval = copy.deepcopy(image_save.detach()).to(device), copy.deepcopy(eval_labs.detach()).to(device) # avoid any unaware modification

                args.lr_net = syn_lr.item()
                _, acc_train, acc_test = evaluate_synset(it_eval, copy.deepcopy(net_eval).to(device), image_syn_eval.to(device), label_syn_eval.to(device), testloader, args, texture=False, train_criterion=criterion)
                accs_test.append(acc_test)
                accs_train.append(acc_train)

            accs_test = np.array(accs_test)
            accs_train = np.array(accs_train)
            acc_test_mean = np.mean(accs_test)
            acc_test_std = np.std(accs_test)

            if acc_test_mean > best_acc[model_eval]:
                best_acc[model_eval] = acc_test_mean
                best_std[model_eval] = acc_test_std
                save_this_it = True
            print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs_test), model_eval, acc_test_mean, acc_test_std))
            wandb.log({'Accuracy/{}'.format(model_eval): acc_test_mean}, step=it)
            wandb.log({'Max_Accuracy/{}'.format(model_eval): best_acc[model_eval]}, step=it)
            wandb.log({'Std/{}'.format(model_eval): acc_test_std}, step=it)
            wandb.log({'Max_Std/{}'.format(model_eval): best_std[model_eval]}, step=it)

    if it in eval_it_pool and (save_this_it or it % 1000 == 0):
        with torch.no_grad():
            image_save = image_syn.cuda()
            save_dir = os.path.join(".", "logged_files", args.dataset, str(args.ipc), args.model, wandb.run.name)

            if not os.path.exists(save_dir):
                os.makedirs(os.path.join(save_dir,'Normal'))

            torch.save(image_save.cpu(), os.path.join(save_dir, 'Normal',"images_{}.pt".format(it)))
            torch.save(label_syn.cpu(), os.path.join(save_dir, 'Normal', "labels_{}.pt".format(it)))
            torch.save(syn_lr.detach().cpu(), os.path.join(save_dir, 'Normal', "lr_{}.pt".format(it)))

            if save_this_it:
                torch.save(image_save.cpu(), os.path.join(save_dir, 'Normal', "images_best.pt".format(it)))
                torch.save(label_syn.cpu(), os.path.join(save_dir, 'Normal', "labels_best.pt".format(it)))
                torch.save(syn_lr.detach().cpu(), os.path.join(save_dir, 'Normal', "lr_best.pt".format(it)))

            wandb.log({"Pixels": wandb.Histogram(torch.nan_to_num(image_syn.detach().cpu()))}, step=it)

            if args.ipc < 50 or args.force_save:
                upsampled = image_save
                if args.dataset != "ImageNet":
                    upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                    upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                wandb.log({"Synthetic_Images": wandb.Image(torch.nan_to_num(grid.detach().cpu()))}, step=it)
                wandb.log({'Synthetic_Pixels': wandb.Histogram(torch.nan_to_num(image_save.detach().cpu()))}, step=it)

                for clip_val in [2.5]:
                    std = torch.std(image_save)
                    mean = torch.mean(image_save)
                    upsampled = torch.clip(image_save, min=mean-clip_val*std, max=mean+clip_val*std)
                    if args.dataset != "ImageNet":
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                    grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                    wandb.log({"Clipped_Synthetic_Images/std_{}".format(clip_val): wandb.Image(torch.nan_to_num(grid.detach().cpu()))}, step=it)

                if args.zca:
                    image_save = image_save.to(args.device)
                    image_save = args.zca_trans.inverse_transform(image_save)
                    image_save.cpu()
                    torch.save(image_save.cpu(), os.path.join(save_dir, 'Normal', "images_zca_{}.pt".format(it)))
                    upsampled = image_save
                    if args.dataset != "ImageNet":
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                    grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                    wandb.log({"Reconstructed_Images": wandb.Image(torch.nan_to_num(grid.detach().cpu()))}, step=it)
                    wandb.log({'Reconstructed_Pixels': wandb.Histogram(torch.nan_to_num(image_save.detach().cpu()))}, step=it)
                    for clip_val in [2.5]:
                        std = torch.std(image_save)
                        mean = torch.mean(image_save)
                        upsampled = torch.clip(image_save, min=mean - clip_val * std, max=mean + clip_val * std)
                        if args.dataset != "ImageNet":
                            upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                            upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                        grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)
                        wandb.log({"Clipped_Reconstructed_Images/std_{}".format(clip_val): wandb.Image(
                            torch.nan_to_num(grid.detach().cpu()))}, step=it)



    wandb.log({"Synthetic_LR": syn_lr.detach().cpu()}, step=it)

    student_net = get_network(args.model, channel, num_classes, im_size, dist=False).to(args.device)  # get a random model

    student_net = ReparamModule(student_net)

    if args.distributed:
        student_net = torch.nn.DataParallel(student_net)

    student_net.train()

    num_params = sum([np.prod(p.size()) for p in (student_net.parameters())])

    if args.load_all:
        expert_trajectory = buffer[np.random.randint(0, len(buffer))]
    else:
        expert_trajectory = buffer[buffer_id[expert_idx]]
        expert_idx += 1
        if expert_idx == len(buffer):
            expert_idx = 0
            file_idx += 1
            if file_idx == len(expert_files):
                file_idx = 0
                random.shuffle(expert_id)
            print("loading file {}".format(expert_files[expert_id[file_idx]]))
            if args.max_files != 1:
                del buffer
                buffer = torch.load(expert_files[expert_id[file_idx]])
            if args.max_experts is not None:
                buffer = buffer[:args.max_experts]
            random.shuffle(buffer_id)

    # Only match easy traj. in the early stage
    if args.Sequential_Generation:
        Upper_Bound = args.current_max_start_epoch + int((args.max_start_epoch-args.current_max_start_epoch) * it/(args.expansion_end_epoch))
        Upper_Bound = min(Upper_Bound, args.max_start_epoch)
    else:
        Upper_Bound = args.max_start_epoch

    start_epoch = np.random.randint(args.min_start_epoch, Upper_Bound)

    starting_params = expert_trajectory[start_epoch]
    target_params = expert_trajectory[start_epoch+args.expert_epochs]
    target_params = torch.cat([p.data.to(args.device).reshape(-1) for p in target_params], 0)
    student_params = [torch.cat([p.data.to(args.device).reshape(-1) for p in starting_params], 0).requires_grad_(True)]
    starting_params = torch.cat([p.data.to(args.device).reshape(-1) for p in starting_params], 0)

    syn_images = image_syn
    y_hat = label_syn

    param_loss_list = []
    param_dist_list = []
    indices_chunks = []




    for step in range(args.syn_steps):
        if not indices_chunks:
            indices = torch.randperm(len(syn_images))
            indices_chunks = list(torch.split(indices, args.batch_syn))

        these_indices = indices_chunks.pop()

        x = syn_images[these_indices]
        this_y = y_hat[these_indices]


        if args.dsa and (not args.no_aug):
            x = DiffAugment(x, args.dsa_strategy, param=args.dsa_param)

        if args.distributed:
            forward_params = student_params[-1].unsqueeze(0).expand(torch.cuda.device_count(), -1)
        else:
            forward_params = student_params[-1]
        x = student_net(x, flat_param=forward_params)
        ce_loss = criterion(x, this_y)

        grad = torch.autograd.grad(ce_loss, student_params[-1], create_graph=True)[0]

        student_params.append(student_params[-1] - syn_lr * grad)

    param_loss = torch.tensor(0.0).to(args.device)
    param_dist = torch.tensor(0.0).to(args.device)

    param_loss += torch.nn.functional.mse_loss(student_params[-1], target_params, reduction="sum")
    param_dist += torch.nn.functional.mse_loss(starting_params, target_params, reduction="sum")

    param_loss_list.append(param_loss)
    param_dist_list.append(param_dist)

    param_loss /= num_params
    param_dist /= num_params

    param_loss /= param_dist

    grand_loss = param_loss

    optimizer_img.zero_grad()
    optimizer_lr.zero_grad()
    optimizer_y.zero_grad()

    grand_loss.backward()

    if grand_loss<=args.threshold:
        optimizer_y.step()
        optimizer_img.step()
        optimizer_lr.step()
    else:
        wandb.log({"falts": start_epoch}, step=it)



    wandb.log({"Grand_Loss": param_loss.detach().cpu(),
                "Start_Epoch": start_epoch})

    for _ in student_params:
        del _

    if it%10 == 0:
        print('%s iter = %04d, loss = %.4f' % (get_time(), it, grand_loss.item()))


wandb.finish()


-------------------------
Evaluation
model_train = ConvNetD5, model_eval = ConvNetD5, iteration = 0
DSA augmentation strategy: 
 color_crop_cutout_flip_scale_rotate
DSA augmentation parameters: 
 {'aug_mode': 'S', 'prob_flip': 0.5, 'ratio_scale': 1.2, 'ratio_rotate': 15.0, 'ratio_crop_pad': 0.125, 'ratio_cutout': 0.5, 'ratio_noise': 0.05, 'brightness': 1.0, 'saturation': 2.0, 'contrast': 0.5}


  0%|          | 0/1001 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1001/1001 [00:45<00:00, 22.23it/s]


[2023-11-27 15:46:07] Evaluate_00: epoch = 1000 train time = 45 s train loss = 0.167540 train acc = 0.0350, test acc = 0.7339
Evaluate 1 random ConvNetD5, mean = 0.7339 std = 0.0000
-------------------------
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_8.pt
[2023-11-27 15:46:15] iter = 0000, loss = 0.9500
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_9.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_7.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_5.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_3.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_0.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_4.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_1.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_2.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_5.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_1.pt
[2023-11-27 15:46:43] iter = 0010, loss = 1.0024
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_9.p

100%|██████████| 1001/1001 [00:46<00:00, 21.35it/s]


[2023-11-27 15:56:32] Evaluate_00: epoch = 1000 train time = 46 s train loss = 0.128539 train acc = 0.0350, test acc = 0.8035
Evaluate 1 random ConvNetD5, mean = 0.8035 std = 0.0000
-------------------------
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_2.pt
[2023-11-27 15:56:37] iter = 0200, loss = 0.9865
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_5.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_3.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_9.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_8.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_1.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_6.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_4.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_7.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_8.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_1.pt
[2023-11-27 15:57:05] iter = 0210, loss = 0.8918
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_5.p

...

100%|██████████| 1001/1001 [00:47<00:00, 21.17it/s]


[2023-11-27 16:27:34] Evaluate_00: epoch = 1000 train time = 47 s train loss = 0.119265 train acc = 0.0350, test acc = 0.7902
Evaluate 1 random ConvNetD5, mean = 0.7902 std = 0.0000
-------------------------
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_5.pt
[2023-11-27 16:27:37] iter = 0800, loss = 0.7841
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_4.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_2.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_0.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_1.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_7.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_9.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_3.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_6.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_3.pt
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_0.pt
[2023-11-27 16:28:06] iter = 0810, loss = 0.8219
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_6.p

100%|██████████| 1001/1001 [00:47<00:00, 21.15it/s]


[2023-11-27 16:37:55] Evaluate_00: epoch = 1000 train time = 47 s train loss = 0.118535 train acc = 0.0350, test acc = 0.7932
Evaluate 1 random ConvNetD5, mean = 0.7932 std = 0.0000
-------------------------
loading file ./buffers/MHIST/ConvNetD5/replay_buffer_2.pt
[2023-11-27 16:38:00] iter = 1000, loss = 0.9575


Accuracy/ConvNetD5,▁█▇▆▇▇
Grand_Loss,█▇█▇▇▆▅▇▇▇▇▆▇▇█▇▇▇▆▆▆▇▇▆▆▇▆▄▇▇▆▄▄▄▁▇▅▅▇█
Max_Accuracy/ConvNetD5,▁█████
Max_Std/ConvNetD5,▁▁▁▁▁▁
Progress,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Start_Epoch,█▅▅▄▇▂▂█▇▇▄▄▆▆▅▅▇▆▄█▄▆▄▇█▇▂▂▆▇▄▁▁▁▁▄▂▅▄▇
Std/ConvNetD5,▁▁▁▁▁▁
Synthetic_LR,▁▁▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
falts,▃▃▁▇▅▅▇█▁▅█▁▇▅▁▅▇██
Accuracy/ConvNetD5,0.79324
Grand_Loss,0.95747
